In [1]:
import mbuild as mb
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import gsd.hoomd
import gsd

In [2]:
mb.__version__

'0.17.0'

In [ ]:
gsd.version()

In [3]:
class CopperLattice(mb.Compound):
    def __init__(self, x, y, z, pillar_height_percent=1/3):
        '''Make sure x and y values are divisible '''
        self.php = pillar_height_percent
        super(CopperLattice, self).__init__()
        spacings = [0.36149, 0.36149, 0.36149]
        angles = [90, 90, 90]
        points = [[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]]
        fcc_lattice = mb.Lattice(lattice_spacing=spacings, angles=angles, lattice_points={'A' : points})
        # define Compound that is a copper atom
        cu = mb.Compound(name='Cu')
        # populate lattice with compounds
        self.add(fcc_lattice.populate(compound_dict={'A' : cu}, x=x, y=y, z=z))

    @property
    def Lx(self):
        return self.box.Lx
        
    @property
    def Ly(self):
        return self.box.Ly
        
    @property
    def Lz(self):
        return self.box.Lz

    @property
    def slab_z_chunk(self):
        return (1-self.php) * self.Lz

    @property
    def surface_thickness(self):
        pass
    
    def remove_region(self, x_range, y_range, z_range):
        remove_particles = []
        for p in self.particles():
            if p.pos[0] >= x_range[0] and p.pos[0] < x_range[1]:
                if p.pos[2] >= z_range[0] and p.pos[2] < z_range[1]:
                    remove_particles.append(p)
        for p in remove_particles:
            self.remove(p)

    def make_two_pillars(self):
        """Remove 3 chunks, leaving 2 pillars"""
        # Left-most region
        x_range = (0, self.Lx / 5)
        x_range2 = (2*self.Lx/5, 3*self.Lx/5)
        x_range3 = (4*self.Lx/5, self.Lx)
        y_range = (0, self.Ly)
        z_range = (self.slab_z_chunk/2, self.Lz - (self.slab_z_chunk/2))
        self.remove_region(x_range, y_range, z_range)
        self.remove_region(x_range2, y_range, z_range)
        self.remove_region(x_range3, y_range, z_range)

    def make_four_pillars(self):
        """Remove 5 chunks, leaving 4 pillars"""
        # Left-most region
        x_range = (0, self.Lx / 9)
        x_range2 = (2*self.Lx/9, 3*self.Lx/9)
        x_range3 = (4*self.Lx/9, 5*self.Lx/9)
        x_range4 = (6*self.Lx/9, 7*self.Lx/9)
        x_range5 = (8*self.Lx/9, self.Lx)
        y_range = (0, self.Ly)
        z_range = (self.slab_z_chunk/2, self.Lz - (self.slab_z_chunk/2))
        self.remove_region(x_range, y_range, z_range)
        self.remove_region(x_range2, y_range, z_range)
        self.remove_region(x_range3, y_range, z_range)
        self.remove_region(x_range4, y_range, z_range)
        self.remove_region(x_range5, y_range, z_range)

    def make_three_pillars(self):
        """Remove 5 chunks, leaving 4 pillars"""
        # Left-most region
        x_range = (0, self.Lx / 9)
        x_range2 = (2*self.Lx/9, 3*self.Lx/9)
        x_range3 = (4*self.Lx/9, 5*self.Lx/9)
        x_range4 = (6*self.Lx/9, 7*self.Lx/9)
        x_range5 = (8*self.Lx/9, self.Lx)
        y_range = (0, self.Ly)
        z_range = (self.slab_z_chunk/2, self.Lz - (self.slab_z_chunk/2))
        self.remove_region(x_range, y_range, z_range)
        self.remove_region(x_range2, y_range, z_range)
        self.remove_region(x_range3, y_range, z_range)
        self.remove_region(x_range4, y_range, z_range)
        self.remove_region(x_range5, y_range, z_range)

In [4]:
lattice = CopperLattice(x=15, y=15, z=10)
lattice.make_two_pillars()
lattice.save('two_pillar.gsd')

In [ ]:
lattice.visualize()
#analysis, how long for an atom to move from one slab to the other (transit time, would require writing own function)
#does self diffusion coefficient change based on pillar addition
#change pillar heights/tall skinny boys, how does that change the self diffusion coefficient
#measure self diffusion coefficient with MSD
#what pillar ratio do we stop seeing bulk properties
#x,y,z = how many times is copper unit cell repeated / system size and pillar size
#first runs: how hot can we go before pillars melt? use a 2 pillar system temp sweep